In [1]:
import wandb
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
wandb.login(key="8a88a8c49d1c2d31b8677fe0b8eb7d3e3a031f83")
api = wandb.Api()


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /export/home/0schindl/.netrc
wandb: Currently logged in as: belaschindler (belaschindler-university-hamburg) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [12]:
sweep_id = "belaschindler-university-hamburg/0schindl-LayUp_sweeps_question1_selection_method/wk4w5q0t"
sweep = api.sweep(sweep_id)
runs = sweep.runs



data = []
for run in runs:
    config = run.config
    summary = run.summary

    dataset = config.get("dataset")
    selection_method = config.get("selection_method")
    mean_acc = summary.get("task_mean/acc")
    run_id = run.id
    state = run.state

    if dataset is not None and selection_method is not None and mean_acc is not None and state == "finished":
        data.append({
            "run_id": run_id,
            "dataset": dataset,
            "selection_method": selection_method,
            "mean_acc": mean_acc
            })

df_sweep = pd.DataFrame(data)
print("\nDataFrame der Sweep-Runs:")
print(df_sweep.head())

# Gruppiere nach Dataset und finde die beste selection_method basierend auf mean_acc (wie zuvor)
best_methods_sweep = df_sweep.loc[df_sweep.groupby("dataset")["mean_acc"].idxmax()]
print("\nBeste Selection Methods pro Dataset im Sweep:")
print(best_methods_sweep)



DataFrame der Sweep-Runs:
     run_id        dataset selection_method  mean_acc
0  otpftybu  omnibenchmark       inv_kl_div  0.557919
1  dps0ppun      imageneta           ws_div  0.391483
2  kjxo0x09  omnibenchmark           around  0.587812
3  93y21l92           cars           ws_div  0.272995
4  fc1z9ff6           cars       inv_kl_div  0.277637

Beste Selection Methods pro Dataset im Sweep:
      run_id        dataset selection_method  mean_acc
5   al08o88u           cars           kl_div  0.279716
49  q9r1xc9a       cifar100       inv_ws_div  0.780800
34  b0dzsjax            cub       inv_ws_div  0.730753
32  2f76fh74  dil_imagenetr       eucld_dist  0.060582
26  i456qxf5      imageneta       inv_kl_div  0.402793
27  e7fey5va      imagenetr       inv_kl_div  0.575949
2   kjxo0x09  omnibenchmark           around  0.587812
10  h9k1tnoy           vtab           ws_div  0.790272


In [14]:
sorted_methods_per_dataset = df_sweep.groupby("dataset").apply(
    lambda x: x.sort_values(by="mean_acc", ascending=False)
)

with pd.option_context('display.max_rows', None):
    print(sorted_methods_per_dataset)
print(sorted_methods_per_dataset.shape)


                    run_id        dataset selection_method  mean_acc
dataset                                                             
cars          5   al08o88u           cars           kl_div  0.279716
              4   fc1z9ff6           cars       inv_kl_div  0.277637
              6   ua1ld836           cars   inv_eucld_dist  0.277278
              7   kvj4knc9           cars       eucld_dist  0.275654
              8   kjh5dj04           cars           around  0.275096
              3   93y21l92           cars           ws_div  0.272995
cifar100      49  q9r1xc9a       cifar100       inv_ws_div  0.780800
              37  vip27gx3       cifar100       inv_kl_div  0.780200
              51  4hjczvft       cifar100       eucld_dist  0.776200
              50  p5xo2f0o       cifar100           kl_div  0.775300
              46  3487s1kp       cifar100   inv_eucld_dist  0.773800
              36  4ww7bxkx       cifar100           ws_div  0.770300
cub           34  b0dzsjax        

In [26]:
fill_value = "N/A"  # Hier kannst du deinen gewünschten Füllwert festlegen

def rank_selection_methods(group):
    """Ordnet die Selection Methods innerhalb einer Dataset-Gruppe nach mean_acc."""
    ranked = group.sort_values(by='mean_acc', ascending=False)['selection_method'].reset_index(drop=True)
    return ranked

ranked_methods = df_sweep.groupby('dataset').apply(rank_selection_methods)

# Erstelle einen neuen DataFrame für die Übersicht mit Füllwerten
overview_ranked = pd.DataFrame()
max_rows = 0
for dataset in ranked_methods.index.get_level_values('dataset').unique():
    methods = ranked_methods[dataset]
    overview_ranked[dataset] = methods.reindex(range(len(methods)), fill_value=fill_value)
    max_rows = max(max_rows, len(methods))

# Setze den Index basierend auf der tatsächlichen Anzahl der Zeilen im DataFrame
overview_ranked.index = [f"{i+1}." for i in range(len(overview_ranked))]

pd.set_option('display.width', 5000)
pd.set_option('display.max_colwidth', None)
print(overview_ranked)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


              cars        cifar100             cub   dil_imagenetr       imageneta       imagenetr omnibenchmark            vtab
1.          kl_div      inv_ws_div      inv_ws_div      eucld_dist      inv_kl_div      inv_kl_div        around          ws_div
2.      inv_kl_div      inv_kl_div          ws_div          ws_div          kl_div          ws_div    inv_kl_div      eucld_dist
3.  inv_eucld_dist      eucld_dist      inv_kl_div      inv_kl_div  inv_eucld_dist  inv_eucld_dist           NaN          around
4.      eucld_dist          kl_div          kl_div          kl_div      inv_ws_div      eucld_dist           NaN          kl_div
5.          around  inv_eucld_dist      eucld_dist          around      eucld_dist      inv_kl_div           NaN      inv_kl_div
6.          ws_div          ws_div  inv_eucld_dist  inv_eucld_dist      eucld_dist          around           NaN  inv_eucld_dist
